In [ ]:
import pandas as pd
import geopandas as gpd
from src.get_data_lycees import get_data_lycees
from src.get_data_biblio import get_data_biblio
from plotnine import *
import folium

pd.set_option('display.width', 120)

In [ ]:
lycees_data = get_data_lycees()

In [ ]:
lycees_data.info()

In [ ]:
(
    lycees_data
    .assign(
        # test = lycees_data['taux_men_gnle'] > -200
    )
    .groupby(["code_region", "libelle_region"]).size()
)

In [ ]:
p = (
    ggplot(
        lycees_data,
        aes(
            x = "ips_voie_gt",
            y = "taux_men_gnle",
            color = "va_men_gnle"
    )
  ) + 
  geom_point() + 
  geom_smooth(method = 'lm') +
  scale_color_gradient2() +
    facet_wrap("libelle_region") +
    theme_bw()
)

# p.save(height=8, width=12, format='svg')
p.save("plot_region.png", height=8, width=12, format='png')

In [ ]:
lycees_idf = lycees_data[lycees_data['code_region'] == "11"]
lycees_idf.columns

In [ ]:
(
    lycees_idf
    .assign(
        # test = lycees_data['taux_men_gnle'] > -200
    )
    .groupby(["code_departement", "libelle_departement"]).size()
)

In [ ]:
center = [lycees_idf['latitude'].mean(), lycees_idf['longitude'].mean()]
m = folium.Map(location=center, zoom_start=12)

tooltip = folium.GeoJsonTooltip(
    fields=['libelle_etablissement', 'presents_gnle', 'taux_reu_gnle', 'taux_men_gnle', 'ips_voie_gt'],
    aliases=[
        'Nom :', 'Nombre de candidats présents :', 'Taux de réussite :', 'Taux de mention',
        'Indice de position sociale :'
    ],
    localize=True
)

style_function = lambda x: {
    'fillColor': 'red',
    'color': 'blue',
    'weight': .5,
    'fillOpacity': 0.0
}

folium.GeoJson(
    lycees_idf,
    style_function=style_function,
    tooltip=tooltip
).add_to(m)
m

In [ ]:
biblio = get_data_biblio()

In [ ]:
(
    biblio
    .assign(
        # test = lycees_data['taux_men_gnle'] > -200
    )
    .groupby(["REG"]).size()
)

In [ ]:
(
    biblio[biblio['REG'] == "11"]
    .assign(
        # test = lycees_data['taux_men_gnle'] > -200
    )
    .groupby(["DEP"]).size()
)

In [ ]:
biblio[biblio['DEP'] == "75"].head()

In [ ]:
biblio_75 = biblio[biblio['DEP'] == "75"].to_crs(lycees_idf.crs)
lycees_75 = lycees_data[lycees_data['code_departement'] == "075"]

In [ ]:
center = [biblio_75.LATITUDE.mean(), biblio_75.LONGITUDE.mean()]
m = folium.Map(
    location=center, 
    zoom_start=12
    )

tooltip = folium.GeoJsonTooltip(
    fields=['NOMRS', 'CATEGORIE', 'LIBCOM'],
    aliases=['Nom :', 'Catégorie : ', 'Commune :'],
    localize=True
)
folium.GeoJson(
    biblio_75,
    tooltip=tooltip, 
    marker=folium.Marker(icon=folium.Icon(color="green")),
).add_to(m)

tooltip = folium.GeoJsonTooltip(
    fields=['libelle_etablissement', 'presents_gnle', 'taux_reu_gnle', 'taux_men_gnle', 'ips_voie_gt'],
    aliases=[
        'Nom :', 'Nombre de candidats présents :', 'Taux de réussite :', 'Taux de mention',
        'Indice de position sociale :'
    ],
    localize=True
)
folium.GeoJson(
    lycees_75,
    tooltip=tooltip, 
    marker=folium.Marker(icon=folium.Icon(color="blue")),
).add_to(m)

m